In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import torch
import gradio as gr

# Initialize available models for text generation
models = {
    "Meta Llama 3.2 1B": "meta-llama/Llama-3.2-1B-Instruct",
    "Samba Lingo Serbian Chat": "sambanovasystems/SambaLingo-Serbian-Chat",
    "Meta Llama 3.2 1B RS": "mradermacher/Llama-3.2-1B-Instruct-RS-GGUF"
};

# Function to load the selected model
def load_model(model_name):
    "Load the specified model."
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False);
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto");
    generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=-1)
    return generator

# Initialize sentence embedding model for retrieval
retrieval_model = SentenceTransformer("sentence-transformers/LaBSE");

# Knowledge base with letter excerpts from Ivo Andrić
knowledge_base = [
    {
        "category": "Knjige i pisma koje je slao Ivo Andric Veri Stojic",
        "content": ("Ivo Andrić je slao knjige i pisma Veri Stojić, ali su se često gubila ili kasnila. "
                    "Ivo Andrić u svom pismu Veri izražava nezadovoljstvo zbog kašnjenja knjige koju je poslao: '...ne mogu da se načudim da niste primili ni knjigu koju Vam je poslala ovdašnja knjižara Fueri, preporučeno, ni moje pismo koje sam nekoliko dana docnije pisao.' "
                    "Ivo Andrić je poslao knjigu 'muzički rečnik' Veri jer je smatrao da će joj biti koristan. "
                    "Ivo Andrić u svom pismu Veri navodi: 'Knjiga koju sam Vam poslao bio je muzički rečnik, mislio sam da će Vam biti koristan i prijatan; urgiraću na pošti.' "
                    "O čestom kašnjenju pisama između Ive Andrića i Vere Stojić svedoči i sledeća rečenica: '...na moja tri pisma nemam od Vas odgovora, i ako se Vi u svakom vašem pismu žalite da Vam ne pišem.' "
                    "Vera je na kraju ipak primila i knjigu i pismu čemu svedoči sledeća rečenica iz pisma Ive Andrića: '...primio sam vaše pismo iz kog vidim da ste, ako i sa zadocnjenjem, ipak sve primili u redu, i pisma i knjigu, i da ste živo i zdravo.' ")   
    },
    {
        "category": "Zdravstveno stanje Ive Andrića",
        "content": ("Ivo Andrić se često žali na svoje zdravlje. Najviše pominje grip i anginu. "
                    "Vezano za svoje zdravlje on kaže: 'Ovde je infernalna vrućina, a ja moram da pijem čajeve i gutam aspirine.' "
                    "U jednom od svojih pisama Ivo Andrić se žali što je zbog lošeg zdravlja propustio narodno veselje: '...od sinoć ležim, sa anginom i groznicom. Nema ništa od našeg narodnog veselja. Nadam se do sutra, najdalje prekosutra, prezdraviti.' "
                    "U jednom pismu Ivo Andrić je opisao svoje zdravstveno stanje ovako: 'Kolebam sa zdravljem, a ova proletnja vremena sa vjetrovima i košavama utiču rđavo na živce, naročito kod ljudi koji ni inače nisu posve kako treba.' "
                    "Ivo Andrić o svom zdravlju navodi i sledeće: '...vrativši se iz Pariza, prebolio (sam) i grip i anginu. Angina je prošla, ali mi je grip još uvek u kostima.'")
    },
    {
        "category": "Boravak Ive Andrića u Mareslju",
        "content": ("Ivo Andrić nije bio zadovoljan svojim boravkom u Marselju. "
                    "On opisuje život u Marselju kao jednoličan. "
                    "Nisu mu prijale temperature u Marselju, što se vidi iz dela pisma u kojem piše: 'Ovde infernalna vrućina...'. "
                    "Nakon povratka u Marselj Ivo Andrić kaže: 'Sad sam opet u Marselju gde život ne pruža mnogo i gde se živi jednolično: između kuće, kancelarije i kavane.' "
                    "Još jedno njegovo viđenje Marselja on iznosi u pismu: 'Vidim dosta cirkusa i ovde. Prvom prilikom čitaću vam svoje utiske i mišljenja iz Francuske. Oni će biti malo drukčiji nego impresije našeg dragog Duke.'")
    }
];


# Encode knowledge base texts for similarity search
embedded_knowledge = retrieval_model.encode([entry["content"] for entry in knowledge_base]);

# Function to find the most relevant text snippet based on the query
def retrieve_snippet(query):
    """Retrieve the most relevant knowledge snippet for a given query."""
    query_embedded = retrieval_model.encode([query]);                               # Encode the query to obtain its embedding
    similarities = retrieval_model.similarity(embedded_knowledge, query_embedded);  # Calculate cosine similarities between the query embedding and the snippet embeddings
    best_match_index = similarities.argmax().item();
    retrieved_entry = knowledge_base[best_match_index];                             # Retrieve the entry snippet with the highest similarity
    return retrieved_entry["content"], retrieved_entry["category"];

# Function to generate responses using RAG (Retrieval-Augmented Generation)
def ask_with_rag(query, model_choice):
    """Generate an answer using RAG by retrieving relevant context from the knowledge base."""
    context, category = retrieve_snippet(query);
    messages = [
        {"role": "system", "content":
                "You are a helpful AI assistant."
                "Provide one Answer ONLY the following query based on the context provided below. "
                "Provide answer in Serbian language."
                "Do not generate or answer any other questions. "
                "Do not make up or infer any information that is not directly stated in the context. "
                "Provide a concise answer."
                f"Based on the category '{category}', answer the question within the provided context:\n{context}."},
        {"role": "user", "content": query},
    ];
    generator = load_model(models[model_choice]);
    response = generator(messages, max_new_tokens=128)[-1]["generated_text"][-1]["content"];
    return context, response;

# Function to generate responses without RAG (free-form response)
def ask_without_rag(query, model_choice):
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant." 
         "Provide an answer to the user's question." 
         "Provide answer in Serbian language."},
        {"role": "user", "content": query},
    ];

    generator = load_model(models[model_choice]);
    response = generator(messages, max_new_tokens=128)[-1]["generated_text"][-1]["content"]
    return response;

# Function to determine whether to use RAG or not
def chat_with_mode(query, use_rag, model_choice):
    if use_rag:
        context, response = ask_with_rag(query, model_choice);
        return f"Kontekst:\n{context}\n Odgovor:\n{response}";
    else:
        response = ask_without_rag(query, model_choice);
        return f"Odgovor:\n{response}";

# Gradio UI Setup
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Chatbot</center></h1>");

    model_choice = gr.Dropdown(list(models.keys()), label="Izaberite željeni model:")    
    
    use_rag = gr.Checkbox(label="Koristi RAG", value=True);

    txt_input = gr.Textbox(show_label=False, placeholder="Ovdje možeš postaviti pitanje...");

    output = gr.Textbox(label="Odgovor", lines=8);

    submit_btn = gr.Button("Pitaj");
    submit_btn.click(chat_with_mode, inputs=[txt_input, use_rag, model_choice], outputs=output);

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
